# ТЗ

In [ ]:
eid - id группы событий (показа и клика к нему) 
t - время 
user - id пользователя 
tn - тип события 
geo -  id региона 
cnethash - хеш от сети класса С, к которой принадлежит ip пользователя
devtype - тип устройства
brotype - тип браузера
bsize - id размера банера
siteid - id канала поставщика трафика
domhash - хеш от домена
urlhash - хеш от урла

В файле данные по одной рекламной кампании. Каждая строка - это клик (tn=1) или показ (tn=0). 
В кампании было накручено много кликов, не являющихся кликами реальных пользователей, осознанно кликавших по рекламе. 
Необходимо выявить такие клики любым удобным способом и составить краткое резюме проделанной работы с объяснением критериев выбора.
При возникновении вопросов относительно данных или выполнения задачи, пишите их контактному лицу по вакансии.

# Как мы можем понять, что клик накручен? 

### 1 Я бы  посмотрел на geo, если большая часть кликов приходит с одного гео, это повод задуматься
### 2 Также стоит посмотреть на chethash, если большая часть кликов приходит с одного chethash, это тоже странно, возможно это прокси
### 3 siteid, если подумать логически, то поставщику трафика как бы выгодно давать нам ботов, чтобы мы смотрели на высокий уровень конверсии и с радостью заказывали трафик у этого поставщика, поэтому я бы также посмотрел на данный фактор

In [262]:
import pandas as pd 
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt 
from statsmodels.distributions.empirical_distribution import ECDF


# Считываем файл 

In [263]:
df = pd.read_csv(r"C:\Users\Sasha\OneDrive\Рабочий стол\тестовое младший аналитик\test_data.tsv", sep = "\t")

In [264]:
df.head()

,eid,t,user,tn,geo,cnethash,devtype,brotype,bsize,siteid,domhash,urlhash
0,347854231724,1050848,21,0,3818162082,8335634595980108828,2,2,1642713892,1732823002,17261330804588317013,16521739535555380904
1,329455126064,694606,36,0,3818162082,2563013194488493467,3,2,2286540757,673473971,8781226038565509786,14273372146021722044
2,329455254926,694659,36,0,3818162082,2563013194488493467,3,2,2286540757,673473971,8781226038565509786,14273372146021722044
3,329455273798,694596,36,0,3818162082,2563013194488493467,3,2,3529439892,673473971,8781226038565509786,14273372146021722044
4,329455293738,694608,36,0,3818162082,2563013194488493467,3,2,390304810,673473971,8781226038565509786,14273372146021722044


# Перед тем как начать работу непосредственно с ТЗ, я хочу немного познакомиться с данными (провести развыдывательный анализ)

In [314]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48672 entries, 0 to 48671
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   eid       48672 non-null  int64 
 1   t         48672 non-null  int64 
 2   user      48672 non-null  int64 
 3   tn        48672 non-null  int64 
 4   geo       48672 non-null  int64 
 5   cnethash  48672 non-null  uint64
 6   devtype   48672 non-null  int64 
 7   brotype   48672 non-null  int64 
 8   bsize     48672 non-null  int64 
 9   siteid    48672 non-null  int64 
 10  domhash   48672 non-null  uint64
 11  urlhash   48672 non-null  uint64
dtypes: int64(9), uint64(3)
memory usage: 4.5 MB


### Сразу заметим что все данные в таблице заполнены и не встречается null значений, плюс нам не нужно менять формат у столбцов, что также упрощает задачу, проверим теперь дубликаты 

In [315]:
# Проверяем дубли по всей таблице 
df[df.duplicated(keep=False)]

,eid,t,user,tn,geo,cnethash,devtype,brotype,bsize,siteid,domhash,urlhash


In [316]:
# Посмотрим дубли по юзерам
df[df.duplicated(subset='user')]

,eid,t,user,tn,geo,cnethash,devtype,brotype,bsize,siteid,domhash,urlhash
2,329455254926,694659,36,0,3818162082,2563013194488493467,3,2,2286540757,673473971,8781226038565509786,14273372146021722044
3,329455273798,694596,36,0,3818162082,2563013194488493467,3,2,3529439892,673473971,8781226038565509786,14273372146021722044
4,329455293738,694608,36,0,3818162082,2563013194488493467,3,2,390304810,673473971,8781226038565509786,14273372146021722044
8,320411433357,515262,148,0,3864597734,9857242284640379126,3,9,3529439892,3312146801,15469158373442651496,18110395686207991474
10,297366904396,16309,162,0,2169085419,8888378963647691173,3,2,4059410016,2854608146,12397643822621642061,17247241689397869523
...,...,...,...,...,...,...,...,...,...,...,...,...
48663,344718117547,980909,99892,0,239476928,17923180725687075424,2,2,4059410016,1732823002,8226252734070597606,501443720760815850
48665,4022514261,100107,99952,0,3954886488,15378092533632864638,2,2,390304810,1732823002,8417199575432949344,6470467525884367406
48666,4023395323,100124,99952,0,3954886488,15378092533632864638,2,2,390304810,1732823002,8417199575432949344,6470467525884367406
48670,323106589352,557496,99995,0,2960729952,15043720182730569642,3,2,3529439892,673473971,17190617809738632834,17743323135665871600


### Делаем вывод что у нас либо есть база юзеров, которым мы показываем нашу рекламу, либо юзер заходя на наш сайт, видит сразу несколько рекламных постов нашего агенства 
### Посмотрим, сколько у нас уникальных юзеров тогда 

In [317]:
df.user.nunique()

4421

### Еще интересно посмотреть процентное соотношение того, сколько у нас конверсия, с учетом того, что была накрутка

In [319]:
df.tn.value_counts(normalize = True)

tn
0    0.968216
1    0.031784
Name: proportion, dtype: float64

# Окей, мы видим, что конверсия 3,18%, теперь наша задача понять, как из этиъ 3,18% выявить накрученные клики 

In [265]:
t = df[df.tn == 1]

In [320]:
t_site_id = t.siteid.value_counts().reset_index(name = "site_1_cnt")
t_site_id

,siteid,site_1_cnt
0,1732823002,1101
1,673473971,211
2,2019279739,167
3,3312146801,53
4,2854608146,10
5,251638358,5


## Можем заметить что много трафика приходит с 1732823002 сайт id, но нужно оценить это в процентах, потому что может быть, что мы закупили рекламу на siteid не по равну 

In [267]:
s = df[df.tn == 0]

In [268]:
s_site_id = s.siteid.value_counts().reset_index(name = "site_0_cnt")

In [269]:
s_site_id = s_site_id.merge(right = t_site_id, on = "siteid", how = "inner")

In [270]:
s_site_id["site_percent"] = (s_site_id["site_1_cnt"] / s_site_id["site_0_cnt"]) * 100

In [271]:
s_site_id.sort_values(by = "site_percent", ascending= False)

,siteid,site_0_cnt,site_1_cnt,site_percent
1,1732823002,18108,1101,6.080186
3,3312146801,1753,53,3.023388
2,673473971,7051,211,2.992483
5,251638358,187,5,2.673797
4,2854608146,1051,10,0.951475
0,2019279739,18933,167,0.882058


# Можем сделать вывод о том, что много трафика идет с 1732823002, 673473971 siteid. Посмотрим что еще может влиять на конверсию

## Посмотрим на domhash

In [272]:
t_domhash = t.domhash.value_counts().sort_values(ascending=False).reset_index(name = "t_domhash_1")

In [273]:
s_domhash = s.domhash.value_counts().sort_values(ascending=False).reset_index(name = "t_domhash_0")

In [274]:
t_domhash = t_domhash.merge(right = s_domhash, on = "domhash", how = "inner")

In [275]:
t_domhash["domhash_percent"] = (t_domhash["t_domhash_1"] / t_domhash["t_domhash_0"]) * 100

In [276]:
t_domhash = t_domhash[(~t_domhash.t_domhash_0.isin(np.array(range(100)))) & (t_domhash.t_domhash_1 > 10)]
t_domhash = t_domhash.sort_values(by = "domhash_percent", ascending=False).reset_index(drop = True)

### Давайте теперь посмотрим как распределены юзеры из 1732823002, 673473971(сайты, который, как мы думаем, дает накрутку) по domhash

In [277]:
n = df[df.siteid.isin([1732823002, 673473971])]
n.head(2)

,eid,t,user,tn,geo,cnethash,devtype,brotype,bsize,siteid,domhash,urlhash
0,347854231724,1050848,21,0,3818162082,8335634595980108828,2,2,1642713892,1732823002,17261330804588317013,16521739535555380904
1,329455126064,694606,36,0,3818162082,2563013194488493467,3,2,2286540757,673473971,8781226038565509786,14273372146021722044


In [278]:
n_domhash_siteid_0 = n[n.tn == 0]
n_domhash_siteid_1 = n[n.tn == 1]


n_domhash_siteid_0_0 = n_domhash_siteid_0.groupby(by = ["siteid", "cnethash"])["tn"].count().reset_index(name = "tn_0")

In [279]:
n_domhash_siteid_1_1 = n_domhash_siteid_1.groupby(by = ["siteid", "cnethash"])["tn"].count().reset_index(name = "tn_1")

In [280]:
n_domhash_siteid_1_1 = n_domhash_siteid_1_1.merge(right = n_domhash_siteid_0_0, on = ["siteid", "cnethash"], how = "inner")

In [281]:
n_domhash_siteid_1_1["percent"] = (n_domhash_siteid_1_1["tn_1"] / n_domhash_siteid_1_1["tn_0"]) * 100

In [282]:
n_domhash_siteid_1_1 = n_domhash_siteid_1_1[(n_domhash_siteid_1_1.tn_0 > 20) & (n_domhash_siteid_1_1.tn_1 > 10)]
n_domhash_siteid_1_1.sort_values(by = "percent", ascending=False).reset_index(drop = True)

,siteid,cnethash,tn_1,tn_0,percent
0,673473971,9767258174204031868,21,28,75.000000
1,1732823002,6143097380029860724,16,58,27.586207
2,1732823002,17332689675803212787,14,75,18.666667
3,1732823002,11520543113519517546,24,144,16.666667
4,1732823002,4844995861121288436,15,101,14.851485
5,1732823002,11027960393376792123,11,86,12.790698
6,1732823002,6174911276659299031,29,231,12.554113
7,1732823002,13889244512852515180,15,135,11.111111
8,1732823002,18231839446737220973,14,149,9.395973
9,1732823002,18433288214847828652,13,139,9.352518


### Мы можем заметить, что трафик, который идет с siteid (673473971) и cnethash (9767258174204031868) в 75% случаев переходит в конверсию
### Также мы можем заметить что много подозрительного трафика идет с siteid (1732823002) и cnethash (6143097380029860724 / 17332689675803212787) в 28% и 18% случаев также переходит в конверсию

# Последнее что мы сделаем, посмотрим на geo пользователей и сравним его. Но тут могут быть несостыковки, так как юзер может использовать vpn или прокси

In [283]:
geo_1 = t.geo.value_counts().sort_values(ascending=False).reset_index(name = "cnt_geo_1")
geo_1 = geo_1[geo_1["cnt_geo_1"] > 10]

In [284]:
geo_0 = s.geo.value_counts().sort_values(ascending=False).reset_index(name = "cnt_geo_0")
geo_0 = geo_0[geo_0["cnt_geo_0"] > 10]

In [285]:
geo_1 = geo_1.merge(right = geo_0, on = "geo", how = "inner")
geo_1["percent"] = (geo_1["cnt_geo_1"] / geo_1["cnt_geo_0"]) * 100

In [286]:
geo_fin = geo_1.sort_values(by = "percent", ascending= False)
geo_fin.head(2)

,geo,cnt_geo_1,cnt_geo_0,percent
34,3330446895,11,99,11.111111
16,1683184212,29,285,10.175439


### Возьмем гео, где процент конверсии более 5 

In [287]:
geo_fin = geo_fin[geo_fin.percent > 5].reset_index(drop = True)
chosen_geo = np.array(geo_fin.geo)
chosen_geo

array([3330446895, 1683184212, 1778249539, 1636627349, 1535909953,
       3371039404, 3031418426,  496411576, 4072314172, 2348398181])

In [288]:
users_from_chosen_sample = df[(df.geo.isin(chosen_geo)) & (df.siteid.isin([1732823002, 673473971])) & (df.domhash.isin(t_domhash.domhash))]
users_from_chosen_sample

,eid,t,user,tn,geo,cnethash,devtype,brotype,bsize,siteid,domhash,urlhash
25,48922405817,1135654,363,0,496411576,618208581979777446,2,2,3529439892,1732823002,7589661441840127586,13552381492480484636
26,48922413278,1135654,363,0,496411576,618208581979777446,2,2,3529439892,1732823002,7589661441840127586,13552381492480484636
27,48922473728,1135655,363,0,496411576,618208581979777446,2,2,4059410016,1732823002,7589661441840127586,13552381492480484636
28,48923341882,1135669,363,0,496411576,618208581979777446,2,2,3529439892,1732823002,7589661441840127586,13552381492480484636
29,48923341882,1135684,363,1,496411576,618208581979777446,2,2,3529439892,1732823002,7589661441840127586,13552381492480484636
...,...,...,...,...,...,...,...,...,...,...,...,...
44755,350543654662,1111507,92078,0,3031418426,6308757286815202786,2,2,3529439892,1732823002,8016818914374733663,1431829537275018691
44756,350543654955,1111507,92078,0,3031418426,6308757286815202786,2,2,390304810,1732823002,8016818914374733663,1431829537275018691
44757,350543655116,1111527,92078,0,3031418426,6308757286815202786,2,2,3529439892,1732823002,8016818914374733663,1431829537275018691
44758,350544674032,1111527,92078,0,3031418426,6308757286815202786,2,2,3529439892,1732823002,8016818914374733663,1431829537275018691


In [292]:
users_from_chosen_sample_0 = users_from_chosen_sample[users_from_chosen_sample.tn == 0]
users_from_chosen_sample_0 = users_from_chosen_sample_0.groupby(["geo","cnethash","siteid"])["tn"].count().reset_index(name = "tn_0")

In [293]:
users_from_chosen_sample_1 = users_from_chosen_sample[users_from_chosen_sample.tn == 1]
users_from_chosen_sample_1 = users_from_chosen_sample_1.groupby(["geo","cnethash","siteid"])["tn"].count().reset_index(name = "tn_1")

In [296]:
users_from_chosen_sample_1 = users_from_chosen_sample_1.merge(right = users_from_chosen_sample_0, on= ["geo","cnethash","siteid"], how = "inner")

In [311]:
g = users_from_chosen_sample_1[(users_from_chosen_sample_1["tn_0"] > 30) & (users_from_chosen_sample_1["tn_1"] > 10)]

In [ ]:
g["percent"] = (g["tn_1"] / g["tn_0"]) * 100

In [313]:
g.sort_values(by = "percent", ascending=False).reset_index(drop = True)

,geo,cnethash,siteid,tn_1,tn_0,percent
0,1683184212,4844995861121288436,1732823002,13,65,20.000000
1,2348398181,11520543113519517546,1732823002,24,143,16.783217
2,496411576,11027960393376792123,1732823002,11,86,12.790698
3,1778249539,6174911276659299031,1732823002,29,228,12.719298


# Выводы

### Мы можем заметить, что трафик, который идет с siteid (673473971) и cnethash (9767258174204031868) в 75% случаев переходит в конверсию
### Также мы можем заметить что много подозрительного трафика идет с siteid (1732823002) и cnethash (6143097380029860724 / 17332689675803212787) в 28% и 18% случаев также переходит в конверсию

### Дополнительно видно что много трафика идет из geo (1683184212 / 2348398181) и cnethash (4844995861121288436 / 11520543113519517546) и siteid(1732823002)